# SREIO 2019 Data Preprocessing fro Transfofrmer Model

In [46]:
import os
import sys
import json
from tqdm import tqdm
from collections import defaultdict
import numpy as np
import random
from IPython import display
from PIL import Image
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
from matplotlib.pyplot import imshow
import math
%matplotlib inline


In [148]:
# this is where I have downlaoded the raw dataset
! ls ../../data/SROIE2019

0325updated.task1train(626p) test.txt
0325updated.task2train(626p) text.task1_2-test（361p)
SroieNERDataset.ipynb        train.txt
task1_2_test(361p)           valid.txt
task3-test 347p) -


In [149]:
DATA_ROOT_PATH = "../../data/SROIE2019" # point this to your local or colab path!

In [49]:
# Get list of all image files with relative path
img_files = os.listdir(DATA_ROOT_PATH + "/0325updated.task1train(626p)")
img_files = list(map(lambda file: DATA_ROOT_PATH + "/0325updated.task1train(626p)/" + file, img_files))
img_files = [file for file in img_files if file.endswith(".jpg")]
img_files = {file.split("/")[-1].split(".")[0] : file for file in img_files}

In [50]:
def get_text_file_dicts(directory):
    """
    List all the files in the directory and filter .txt files
    
    return:
        dict : {file_name_without_extension : relative_path}
    """
    res = defaultdict(None)
    all_files = os.listdir(directory)
    text_files = [file for file in all_files if file.endswith(".txt")]
    text_files = list(map(lambda file: directory + "/" + file, text_files))
    for file in text_files:
        key = file.split("/")[-1].split(".")[0] # extract file name
        res[key]= file
            
    return res

In [51]:
def normalize_bbox(bbox, file_path):
    image = Image.open(file_path)

    width, height = image.size

#     return [
#          int(1000 * (bbox[0] / width)),
#          int(1000 * (bbox[1] / height)),
#          int(1000 * (bbox[2] / width)),
#          int(1000 * (bbox[3] / height)),
#      ]
    return [
         math.ceil(1000 * (bbox[0] / width)),
         math.ceil(1000 * (bbox[1] / height)),
         math.ceil(1000 * (bbox[2] / width)),
         math.ceil(1000 * (bbox[3] / height)),
     ]

normalize_bbox([457, 156, 625, 193], '../../data/SROIE2019/0325updated.task1train(626p)/X51008123451.jpg'), normalize_bbox([160, 1173, 554, 1212], '../../data/SROIE2019/0325updated.task1train(626p)/X51008123451.jpg')

([611, 123, 836, 152], [214, 923, 741, 954])

In [52]:
def unnormalize_bbox(bbox, file_path):
    # [457, 156, 625, 193] -> [611, 123, 836, 152]
    # [611, 123, 836, 152] -> [457, 156, 625, 193]
    image = Image.open(file_path)
    width, height = image.size

    return [
         math.floor(bbox[0] / 1000 * width),
         math.floor(bbox[1] / 1000 * height),
         math.floor(bbox[2] / 1000 * width),
         math.floor(bbox[3] / 1000 * height),
     ]

print(unnormalize_bbox((611, 123, 836, 152), '../../data/SROIE2019/0325updated.task1train(626p)/X51008123451.jpg'),
unnormalize_bbox([214, 923, 741, 954], '../../data/SROIE2019/0325updated.task1train(626p)/X51008123451.jpg'))

[457, 156, 625, 193] [160, 1173, 554, 1212]


In [53]:
def get_bbox_line(line):
    xycoords = line.split(",")[:8]
    x1, y1, x2, y2, x3, y3, x4, y4 = xycoords
    return [int(x1), int(y1), int(x3), int(y3)]
    

In [54]:
def extract(ocr_text_file, ie_text_file, image_file):
    """
    Match the OCR text with key value pair from infromation extraction task
    """
    
    if ocr_text_file is None or ie_text_file is None or image_file is None:
        return None
    
    if not os.path.exists(ocr_text_file) or not os.path.exists(ie_text_file) or not os.path.exists(image_file):
        return None
    
    ocr_text_file_lines = open(ocr_text_file).readlines()
    # extract bbox !!WARNING : Dataset OCR are for whole line and not for individual token!!
    bbox = list(map(lambda line: get_bbox_line(line), ocr_text_file_lines))
    bbox = list(map(lambda xy: normalize_bbox(xy, image_file), bbox))
    # extract the text from the line
    ocr_text_file_lines = list(map(lambda line: " ".join(line.split(",")[8:]).strip(), ocr_text_file_lines))
    # add default tag O to each line
    token_2_tag = {line : 'O' for line in ocr_text_file_lines}
    bbox_for_ner_tags = {line : xy for line, xy in zip(ocr_text_file_lines, bbox)}
    
    
    ie_text_file_lines = open(ie_text_file).readlines()
    ie_text_file_lines = "".join(ie_text_file_lines)
    # read the data as dict
    ie_data = json.loads(ie_text_file_lines)
    
    # print(ie_data)
    res = {}
    
    for ocr_text, v in token_2_tag.items():
        for tag, ner_text in ie_data.items():
            ner_text = ner_text.replace(",", "")
            
            # omit empty string lines and length less than 2
            if ocr_text != '' and len(ocr_text) > 2 and (ocr_text in ner_text or ner_text in ocr_text):
                token_2_tag[ocr_text] = tag
             
            # For those lines which have multiple tokens
            num_matches = 0 
            for token in ocr_text.split(" "):
                if token != '' and token in ner_text:
                    num_matches += 1
                    
            # make sure we find atleast 2 matches per line to tag that line with respective tag        
            if num_matches > 2:
                token_2_tag[ocr_text] = tag
                    
    return token_2_tag, bbox_for_ner_tags
    

In [55]:
def tagged_data_to_conll(ner_data: dict, bbox_data: dict):
    """
    Given dict of lines and tags, convert it into CoNLL format
    """
    if ner_data is None or bbox_data is None:
        return []
    
    res = []
    
    for text, tag in ner_data.items():
        bbox = bbox_data[text]
        for token in text.split(" "):
            res.append((token, tag.replace("\n", ""), bbox))
    return res

In [56]:

def draw_bbox(img_file, conll_data):
    img = Image.open(img_file)#.convert("RGBA")
#     img.putalpha(127)
    draw = ImageDraw.Draw(img)
    for token, tag, bbox in conll_data:
        if tag != 'O':
            bbox = unnormalize_bbox(bbox, img_file)
            draw.rectangle(bbox, outline='red', width=1)
            draw.text((bbox[0], bbox[1]), tag, font=ImageFont.truetype("Roboto-Light.ttf", 30), stroke_width=0, fill='red', spacing=10)
    
    if os.path.exists('/tmp/temp124.jpg'):
        os.remove('/tmp/temp124.jpg')
    img.save("/tmp/temp124.jpg")
    return display.Image("/tmp/temp124.jpg")


In [57]:
train_ocr_text_files_dict = get_text_file_dicts(DATA_ROOT_PATH + "/0325updated.task1train(626p)")
train_ie_text_files_dict = get_text_file_dicts(DATA_ROOT_PATH + "/0325updated.task2train(626p)")
files = sorted(train_ocr_text_files_dict.keys())

In [147]:
# key = 'X51008123451' #random.choice(list(train_ocr_text_files_dict.keys()))
key = random.choice(list(train_ocr_text_files_dict.keys()))
ocr_text_file = train_ocr_text_files_dict.get(key)
ie_text_file = train_ie_text_files_dict.get(key)
img_file = img_files.get(key.replace(".txt", ".jpg"))

if img_file is None:
    print("Image file not found") 
    
print(img_file, os.path.exists(img_file))
print("\n")

token_2_tag, bbox_for_ner_tags = extract(ocr_text_file, ie_text_file, img_file)

print("\n\n")    
res = tagged_data_to_conll(token_2_tag, bbox_for_ner_tags)

# print(res)

# display.Image(img_file)

draw_bbox(img_file, res)

../../data/SROIE2019/0325updated.task1train(626p)/X51007339651.jpg True







In [125]:
def create_data_file(file_path, files):
    res = []
    for file in tqdm(files):
        ocr_text_file = train_ocr_text_files_dict.get(file)
        ie_text_file = train_ie_text_files_dict.get(file)
        img_file = img_files.get(key.replace(".txt", ".jpg"))
        if None in [ocr_text_file, ie_text_file, img_file]:
            continue
        data = tagged_data_to_conll(*extract(ocr_text_file, ie_text_file, img_file))
        if data:
            res.append(data)

    with open(file_path, "w") as file:
        for lines in res:
            for line in lines:
                if len(line) <= 1:
                    continue 
                line = list(line) # convert tuples to list
                line[2] = map(lambda t: str(t), line[2]) # map all xy coords to str
                line[2] = ",".join(line[2]) # convert list to string delimited by ','
                file.write(" ".join(list(line)))
                file.write("\n")
            file.write("\n")

In [126]:
! mkdir version1

mkdir: version1: File exists


# Prepare Datase

In [127]:
train_files = files[:700]
create_data_file(file_path="version1/train.txt", files=train_files)

100%|██████████| 700/700 [00:04<00:00, 151.67it/s]


In [128]:
valid_files = files[700:750]
create_data_file(file_path="version1/valid.txt", files=valid_files)

100%|██████████| 50/50 [00:00<00:00, 123.58it/s]


In [129]:
test_files = files[750:]
create_data_file(file_path="version1/test.txt", files=test_files)

100%|██████████| 85/85 [00:00<00:00, 141.11it/s]


In [132]:
def validate(file_path):
    tags = set()
    with open(file_path) as file:
        for line in file.readlines():
            if len(line) == 1:
                continue
            tags.add(line.strip().split(" ")[-2])

    print(tags)

In [133]:
validate("version1/train.txt")

{'O', 'date', 'company', 'address', 'total'}
